In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
#https://m2.ru/moskva/nedvizhimost/kupit-kvartiru/?districtIds=103&districtIds=81&districtIds=182&districtIds=218&districtIds=15&districtIds=60&districtIds=164&districtIds=37&districtIds={page_num}

In [48]:

prices = np.array([], dtype = int)
area = np.array([], dtype = float)
rooms = np.array([])
floor = np.array([])
price_meters = np.array([], dtype = int)
districts = np.array([], dtype = str)
time = np.array([])
metro = np.array([])
seller = np.array([])

for page_num in range(1, 251): 
    url = f"https://m2.ru/moskva/nedvizhimost/kupit-kvartiru/new/?districtIds=103&districtIds=81&districtIds=182&districtIds=218&districtIds=15&districtIds=60&districtIds=164&districtIds=37&districtIds={page_num}"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for district_elem in soup.find_all('div', {'class': 'ClClickableAddress__links'}):
        addresses = district_elem.text.strip()
        district = addresses.split('\n')[0]
        districts = np.append(districts, district.replace('\xa0', '').replace('•', ',').replace(",", ".").replace('.', ',', 5).replace('до центра ', '').replace(' км', ''))
    
    #находим все элементы с ценами и добавляем их в список
    for price_elem in soup.find_all('span', {'itemprop': 'price'}):
        price = price_elem['content']
        prices = np.append(prices, int(price))
        
    for info_elem in soup.find_all('div', {'class': 'fonts-module__promo-subheader-alone___4BOzH fonts-module__bold___2Zct2'}):
        info = info_elem.text.strip().split(',')
        if len(info) == 4:
            info = ['.'.join(info[:2])] + info[2:]
        area = np.append(area, float(info[0].replace('\xa0м²', '')))
        rooms = np.append(rooms, (info[1].replace('-комн. квартира', '').strip()))
        floor = np.append(floor, (info[2].replace('\xa0этаж', '').strip()))

    for price_meter_elem in soup.find_all('div', {'class': 'color-primitives-module__black-400___re9Yh fonts-module__primary___2PNSt'}):
        price_meter_str = price_meter_elem.text.replace('&nbsp;', '').replace('₽/м²','')
        price_meters = np.append(price_meters, int(price_meter_str.replace('\xa0', '').replace(' ', '')))
        
    
    for time_elem in soup.find_all('div', {'class': 'colors-named-module__secondary___2WPH_ fonts-module__primary___2PNSt'}):
        timing = time_elem.text.strip()
        time = np.append(time, int(timing.replace(' мин.', '')))
        
    for seller_elem in soup.find_all('div', {'class': 'colors-named-module__secondary___2WPH_ fonts-module__small-alone___38xTq AuthorBadge__overflow'}):
        sellers = seller_elem.text.strip()
        seller = np.append(seller, (sellers.split())[0])
        

In [50]:
prices
len(prices)

5000

In [51]:
area
len(area)

5000

In [52]:
rooms
len(rooms)

5000

In [53]:
rooms = np.where(rooms == 'студия', '0', rooms)
rooms = rooms.astype(np.int64)
rooms

array([1, 1, 2, ..., 1, 1, 0], dtype=int64)

In [54]:
floor
len(floor)

5000

In [55]:
parts = np.char.partition(floor, '/')
floor_live = (parts[:, 0]).astype(np.int64)
display(floor_live)
floor_total = (parts[:, -1]).astype(np.int64)
display(floor_total)

array([24, 15,  7, ..., 20,  9, 32], dtype=int64)

array([38, 33, 15, ..., 33, 33, 33], dtype=int64)

In [56]:
price_meters

array([324674, 270912, 309778, ..., 243040, 241374, 343776])

In [57]:
districts

array(['Москва,ЮВАО,Южнопортовый район,1-й Дубровский проезд,78/14с1,5',
       'Москва,ВАО,Метрогородок,Открытое шоссе,18с1,10.5',
       'Москва,СВАО,Бабушкинский район,Кольская улица,1,12', ...,
       'Москва,ЮАО,Чертаново Южное,улица Газопровод,вл6Г,18.5',
       'Москва,ЮАО,Чертаново Южное,улица Газопровод,вл6Г,18.5',
       'Москва,ЮАО,Нагатино-Садовники,2-й Нагатинский проезд,2к3,8.5'],
      dtype='<U84')

In [58]:
okrugi = np.array([i.split(',')[1] for i in districts])
okrugi

array(['ЮВАО', 'ВАО', 'СВАО', ..., 'ЮАО', 'ЮАО', 'ЮАО'], dtype='<U12')

In [59]:
raion = np.array([i.split(',')[2] for i in districts])
raion[raion == 'Судостроительная улица'] = 'Нагатино-Садовники' #почему то по этой улице в адресах нет района, пробил в гугле - это нагаитно садовники
raion
len(raion)

5000

In [60]:
rasstoyanie = np.array([(i.split(','))[-1] for i in districts])
rasstoyanie 

array(['5', '10.5', '12', ..., '18.5', '18.5', '8.5'], dtype='<U4')

In [61]:
def is_float(value): #https://stackoverflow.com/questions/354038/how-do-i-check-if-a-string-represents-a-number-float-or-int
    try:
        float(value)
        return True
    except ValueError:
        return False
v_is_float = np.vectorize(is_float)

#в процессе выяснилось, что есть корявые объявление, где нормальным образом не написано расстояние до центра, 
#поэтому вызникали всякие странные значения по типу "25.13.8", которые я решил просто за NaNить

rasstoyanie = np.where(rasstoyanie == 'В центре', '0', rasstoyanie)
rasstoyanie[v_is_float(rasstoyanie)] = np.array(rasstoyanie[v_is_float(rasstoyanie)], dtype=np.float64)
rasstoyanie[~v_is_float(rasstoyanie)] = None
rasstoyanie

array(['5.0', '10.5', '12.0', ..., '18.5', '18.5', '8.5'], dtype='<U4')

In [45]:
time

array([ 9., 19.,  7., 11.,  7.,  7.,  7., 19.,  9.,  9., 11., 19., 19.,
       12.,  7., 39., 37., 11.,  9.,  7.,  9., 19.,  7.,  7.,  7.,  7.,
       19.,  9.,  9., 19., 19.,  7., 39.,  9.,  7.,  9., 13., 20., 14.,
       20.,  9., 19.,  7.,  7.,  7.,  7., 19.,  9.,  9., 19., 19.,  7.,
       39.,  9.,  7.,  9., 13., 20., 14., 20.,  9., 19.,  7.,  7.,  7.,
        7., 19.,  9.,  9., 19., 19.,  7., 39.,  9.,  7.,  9., 13., 20.,
       14., 20.,  9., 19.,  7., 19.,  7.,  7.,  7., 19.,  9.,  9., 19.,
       19., 19., 12.,  7., 39., 19.,  9.,  7., 12.,  9., 19.,  7.,  7.,
        7.,  7., 19.,  9.,  9., 19., 19.,  7., 39.,  9.,  7.,  9., 13.,
       20., 14., 20.,  9., 19.,  7., 11.,  7.,  7.,  7., 19.,  9.,  9.,
       11., 19., 19., 12.,  7., 39., 11.,  9.,  7., 12.,  9., 19.,  7.,
        7.,  7.,  7., 19.,  9.,  9., 19., 19.,  7., 39.,  9.,  7.,  9.,
       13., 20., 14., 20.,  9., 19.,  7., 12.,  7.,  7.,  7., 19.,  9.,
        9., 12., 19., 19., 19.,  7., 39., 12.,  9.,  7., 19.,  9

In [62]:
len(seller[seller=='Застройщик'])

5000

In [63]:
df = pd.DataFrame({'Price_Total': prices, 'Area': area, 'Price_Meter': price_meters, 'Rooms': rooms,
                   'Floor_Live': floor_live, 'Floor_Total': floor_total, 'Zone': okrugi, 'Neighborhood': raion, 'Distance': rasstoyanie,
                  'Time': time})
df.tail(20)

,Price_Total,Area,Price_Meter,Rooms,Floor_Live,Floor_Total,Zone,Neighborhood,Distance,Time
4980,10594407,34.2,309778,2,7,15,СВАО,Бабушкинский район,12.0,11.0
4981,11278620,36.0,313208,1,19,38,ЮВАО,Южнопортовый район,5.0,9.0
4982,11878041,29.8,398592,0,7,32,ЗАО,Филёвский Парк,7.3,9.0
4983,9353943,36.1,259112,1,31,33,ЮАО,Чертаново Южное,18.5,7.0
4984,8484026,24.4,347706,0,19,21,ЗАО,Очаково-Матвеевское,12.1,31.0
4985,8296707,34.2,242452,1,13,33,ЮАО,Чертаново Южное,18.5,7.0
4986,6847017,24.0,285888,0,16,33,ЮВАО,Рязанский район,9.0,15.0
4987,13086430,42.5,307916,1,32,38,ЮВАО,Южнопортовый район,5.0,9.0
4988,21542497,76.8,280684,3,10,21,ЗАО,Очаково-Матвеевское,12.2,31.0
4989,14025503,65.6,213836,2,20,20,ЮАО,Чертаново Южное,18.5,7.0


In [64]:
df.replace('nan', np.nan, inplace=True) #nan не распознавался как NaN, а распознавался как строка, хз почему...
df = df.dropna()

In [65]:
df

,Price_Total,Area,Price_Meter,Rooms,Floor_Live,Floor_Total,Zone,Neighborhood,Distance,Time
0,11850601,36.5,324674,1,24,38,ЮВАО,Южнопортовый район,5.0,9.0
1,9265190,34.2,270912,1,15,33,ВАО,Метрогородок,10.5,12.0
2,10594407,34.2,309778,2,7,15,СВАО,Бабушкинский район,12.0,11.0
3,9353943,36.1,259112,1,31,33,ЮАО,Чертаново Южное,18.5,7.0
4,8484026,24.4,347706,0,19,21,ЗАО,Очаково-Матвеевское,12.1,9.0
...,...,...,...,...,...,...,...,...,...,...
4995,15009300,45.9,327000,2,6,29,ЮЗАО,Черёмушки,11.2,14.0
4996,9764700,26.9,363000,1,22,29,ЮЗАО,Черёмушки,11.2,14.0
4997,8793187,36.2,243040,1,20,33,ЮАО,Чертаново Южное,18.5,7.0
4998,8226025,34.1,241374,1,9,33,ЮАО,Чертаново Южное,18.5,7.0


In [66]:
df.to_csv('our_project.csv', index=False)
